In [1]:
import os
import pandas as pd
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from leafmap.common import evaluate_model, plot_actual_vs_predicted, download_file
import leafmap.maplibregl as leafmap

In [2]:
zhvi= 'https://github.com/opengeos/datasets/releases/download/us/zillow_home_value_index_by_county.csv'
zhvo_file = 'zillow_home_value_index_by_county.csv'
if not os.path.exists(zhvo_file):
    download_file(zhvi, zhvo_file)

In [3]:
zhvi_df= pd.read_csv(zhvo_file, dtype={"StateCodeFIPS": str, "MunicipalCodeFIPS": str})
zhvi_df.index= "geoId/" + zhvi_df["StateCodeFIPS"] + zhvi_df["MunicipalCodeFIPS"]
zhvi_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31
geoId/06037,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",06,037,206685.940073,...,848100.109098,851249.814267,853328.259951,856958.247702,862129.887015,868712.125738,873819.306305,877921.805494,881313.863788,880547.372613
geoId/17031,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,031,145737.060609,...,297797.137247,299609.927472,300652.787054,301303.254592,302118.600852,302781.699599,303133.869223,303523.016515,304181.858348,305000.265124
geoId/48201,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,109327.540062,...,282689.212331,283279.667076,283161.721769,282792.559444,282454.169156,282239.614216,281873.680565,281423.466788,281191.554410,281000.895149
geoId/04013,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",04,013,142829.577207,...,467600.239765,468996.512594,469164.401050,468562.421846,467451.263586,466509.197532,465782.610071,465120.188374,464544.544452,463613.340231
geoId/06073,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",06,073,212384.876155,...,918694.464981,927384.144048,931835.812500,933267.733074,932833.552309,932389.735948,931683.464702,931874.623336,932380.083872,932486.305190


In [4]:
county_geojson= '/home/zyang91/Desktop/us/county.geojson'

In [5]:
county_gdf = gpd.read_file(county_geojson)
county_gdf.set_index("place", inplace=True)
county_gdf.head()

,index,geometry
place,,
geoId/29510,0,"POLYGON ((-90.31662 38.60398, -90.32008 38.593..."
geoId/32510,1,"POLYGON ((-120.00431 39.16561, -120.00297 39.1..."
geoId/51580,2,"POLYGON ((-80.00796 37.76462, -80.0085 37.7611..."
geoId/51530,3,"POLYGON ((-79.38112 37.74236, -79.38063 37.740..."
geoId/51678,4,"POLYGON ((-79.45401 37.79249, -79.46155 37.785..."


In [6]:
df= zhvi_df.join(county_gdf)
df

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,index,geometry
geoId/06037,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",06,037,206685.940073,...,853328.259951,856958.247702,862129.887015,868712.125738,873819.306305,877921.805494,881313.863788,880547.372613,218.0,"MULTIPOLYGON (((-118.67855 33.02634, -118.6748..."
geoId/17031,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,031,145737.060609,...,300652.787054,301303.254592,302118.600852,302781.699599,303133.869223,303523.016515,304181.858348,305000.265124,635.0,"POLYGON ((-88.26308 42.06686, -88.26272 41.986..."
geoId/48201,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,109327.540062,...,283161.721769,282792.559444,282454.169156,282239.614216,281873.680565,281423.466788,281191.554410,281000.895149,2362.0,"POLYGON ((-95.96052 30.16368, -95.92658 30.068..."
geoId/04013,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",04,013,142829.577207,...,469164.401050,468562.421846,467451.263586,466509.197532,465782.610071,465120.188374,464544.544452,463613.340231,88.0,"POLYGON ((-113.33376 33.99923, -113.33389 32.5..."
geoId/06073,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",06,073,212384.876155,...,931835.812500,933267.733074,932833.552309,932389.735948,931683.464702,931874.623336,932380.083872,932486.305190,212.0,"POLYGON ((-117.61109 33.33401, -117.57155 33.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
geoId/31007,846,3206,Banner County,county,NE,NE,"Scottsbluff, NE",31,007,NaN,...,321838.022818,323894.525181,330340.241603,340048.056447,348255.690352,351834.125302,356282.247220,361271.153620,1577.0,"POLYGON ((-104.0527 41.69797, -104.0523 41.393..."
geoId/49009,1648,3207,Daggett County,county,UT,UT,NaN,49,009,NaN,...,308761.377915,306617.625821,306556.018166,308889.348086,313334.046432,316926.936602,320636.922888,323907.317207,2519.0,"POLYGON ((-110.00028 40.81368, -109.99332 40.8..."
geoId/31171,1432,3208,Thomas County,county,NE,NE,NaN,31,171,NaN,...,160305.518515,160113.204884,160637.460741,161362.378182,162603.377093,162802.298298,162976.347349,162721.311102,1586.0,"POLYGON ((-100.84611 42.08818, -100.84436 41.9..."
geoId/31117,2794,3212,McPherson County,county,NE,NE,"North Platte, NE",31,117,NaN,...,288823.914423,291261.158616,298216.019609,308135.413909,317357.571317,322473.200324,328341.338278,331465.598015,1574.0,"POLYGON ((-101.40612 41.74321, -101.40724 41.3..."


In [7]:
zhvi_gdf = gpd.GeoDataFrame(df, geometry='geometry')
zhvi_gdf.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,index,geometry
geoId/06037,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",06,037,206685.940073,...,853328.259951,856958.247702,862129.887015,868712.125738,873819.306305,877921.805494,881313.863788,880547.372613,218.0,"MULTIPOLYGON (((-118.67855 33.02634, -118.6748..."
geoId/17031,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,031,145737.060609,...,300652.787054,301303.254592,302118.600852,302781.699599,303133.869223,303523.016515,304181.858348,305000.265124,635.0,"POLYGON ((-88.26308 42.06686, -88.26272 41.986..."
geoId/48201,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,109327.540062,...,283161.721769,282792.559444,282454.169156,282239.614216,281873.680565,281423.466788,281191.554410,281000.895149,2362.0,"POLYGON ((-95.96052 30.16368, -95.92658 30.068..."
geoId/04013,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",04,013,142829.577207,...,469164.401050,468562.421846,467451.263586,466509.197532,465782.610071,465120.188374,464544.544452,463613.340231,88.0,"POLYGON ((-113.33376 33.99923, -113.33389 32.5..."
geoId/06073,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",06,073,212384.876155,...,931835.812500,933267.733074,932833.552309,932389.735948,931683.464702,931874.623336,932380.083872,932486.305190,212.0,"POLYGON ((-117.61109 33.33401, -117.57155 33.3..."


In [8]:
column= "2024-10-31"
gdf= zhvi_gdf[["RegionName","State", column, "geometry"]]
gdf.head()

,RegionName,State,2024-10-31,geometry
geoId/06037,Los Angeles County,CA,873819.306305,"MULTIPOLYGON (((-118.67855 33.02634, -118.6748..."
geoId/17031,Cook County,IL,303133.869223,"POLYGON ((-88.26308 42.06686, -88.26272 41.986..."
geoId/48201,Harris County,TX,281873.680565,"POLYGON ((-95.96052 30.16368, -95.92658 30.068..."
geoId/04013,Maricopa County,AZ,465782.610071,"POLYGON ((-113.33376 33.99923, -113.33389 32.5..."
geoId/06073,San Diego County,CA,931683.464702,"POLYGON ((-117.61109 33.33401, -117.57155 33.3..."


In [9]:
m=leafmap.Map(style="liberty")
m.add_data(
    gdf,
    cmap="Blues",
    column=column,
    legend_title="Zillow Home Median Home Value",
    name="Zillow Home Median Home Value",
)
m.add_layer_control()
m

Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

In [10]:
m= leafmap.Map(style="liberty", pitch= 60)
m.add_data(
    gdf,
    cmap="Blues",
    column=column,
    extrude=True,
    scale_factor= 3,
    legend_title="Zillow Home Median Home Value",
    name="Zillow Home Median Home Value",
)
m.add_layer_control()
m

Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…